###Relu

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import os
from time import time

###check point 설정

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### 데이터 로드 및 전처리 과정

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] axis=3 이라고 해도됨
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] [batch_size,height,width,channel]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10] , onehotencoding
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

###모델 함수

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
#class.V
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential() #list자료구조 타입
        self.model.add(flatten()) #fc layer 쓸꺼기 때문에 바꿔줌

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x


In [7]:
#func.V
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

###데이터 정의 및 하이퍼 파라미터

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices ((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

11490434/11490434 [==============================] - 3s 0us/step


2022-11-14 15:59:00.690609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


###모델 정의 및 옵피마이저

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

###학습시키기

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.5012, train_loss: 2.17186308, train_accuracy: 0.2969, test_Accuracy: 0.2039
Epoch: [ 0] [    1/  468] time: 0.5879, train_loss: 2.13438082, train_accuracy: 0.4062, test_Accuracy: 0.3362
Epoch: [ 0] [    2/  468] time: 0.6760, train_loss: 2.11364222, train_accuracy: 0.3906, test_Accuracy: 0.4191
Epoch: [ 0] [    3/  468] time: 0.7697, train_loss: 1.99630046, train_accuracy: 0.5547, test_Accuracy: 0.4954
Epoch: [ 0] [    4/  468] time: 0.8614, train_loss: 1.93817854, train_accuracy: 0.5781, test_Accuracy: 0.5981
Epoch: [ 0] [    5/  468] time: 0.9535, train_loss: 1.82669544, train_accuracy: 0.6953, test_Accuracy: 0.6733
Epoch: [ 0] [    6/  468] time: 1.0427, train_loss: 1.75400329, train_accuracy: 0.7109, test_Accuracy: 0.7032
Epoch: [ 0] [    7/  468] time: 1.1275, train_loss: 1.61542606, train_accuracy: 0.8203, test_Accuracy: 0.7094
Epoch: [ 0] [    8/  468] time: 1.2107,

Epoch: [ 0] [   75/  468] time: 7.3871, train_loss: 0.39713788, train_accuracy: 0.8984, test_Accuracy: 0.8955
Epoch: [ 0] [   76/  468] time: 7.4763, train_loss: 0.23487565, train_accuracy: 0.9062, test_Accuracy: 0.8958
Epoch: [ 0] [   77/  468] time: 7.5663, train_loss: 0.28766891, train_accuracy: 0.8828, test_Accuracy: 0.9014
Epoch: [ 0] [   78/  468] time: 7.6562, train_loss: 0.25144169, train_accuracy: 0.9297, test_Accuracy: 0.9107
Epoch: [ 0] [   79/  468] time: 7.7426, train_loss: 0.46445322, train_accuracy: 0.8750, test_Accuracy: 0.9170
Epoch: [ 0] [   80/  468] time: 7.8293, train_loss: 0.33946949, train_accuracy: 0.8828, test_Accuracy: 0.9170
Epoch: [ 0] [   81/  468] time: 7.9193, train_loss: 0.29622343, train_accuracy: 0.9375, test_Accuracy: 0.9124
Epoch: [ 0] [   82/  468] time: 8.0076, train_loss: 0.23719591, train_accuracy: 0.9297, test_Accuracy: 0.9089
Epoch: [ 0] [   83/  468] time: 8.0974, train_loss: 0.35545999, train_accuracy: 0.8828, test_Accuracy: 0.9082
Epoch: [ 0

Epoch: [ 0] [  150/  468] time: 14.2086, train_loss: 0.24271187, train_accuracy: 0.9219, test_Accuracy: 0.9277
Epoch: [ 0] [  151/  468] time: 14.2952, train_loss: 0.12989327, train_accuracy: 0.9688, test_Accuracy: 0.9281
Epoch: [ 0] [  152/  468] time: 14.3806, train_loss: 0.30649033, train_accuracy: 0.9062, test_Accuracy: 0.9293
Epoch: [ 0] [  153/  468] time: 14.4679, train_loss: 0.22388302, train_accuracy: 0.9219, test_Accuracy: 0.9325
Epoch: [ 0] [  154/  468] time: 14.5579, train_loss: 0.32642668, train_accuracy: 0.9219, test_Accuracy: 0.9343
Epoch: [ 0] [  155/  468] time: 14.6443, train_loss: 0.23854570, train_accuracy: 0.9453, test_Accuracy: 0.9359
Epoch: [ 0] [  156/  468] time: 14.7314, train_loss: 0.24339812, train_accuracy: 0.9297, test_Accuracy: 0.9362
Epoch: [ 0] [  157/  468] time: 14.8164, train_loss: 0.26735684, train_accuracy: 0.9062, test_Accuracy: 0.9368
Epoch: [ 0] [  158/  468] time: 14.9028, train_loss: 0.17396173, train_accuracy: 0.9297, test_Accuracy: 0.9369
E

Epoch: [ 0] [  225/  468] time: 20.7928, train_loss: 0.32288361, train_accuracy: 0.9297, test_Accuracy: 0.9410
Epoch: [ 0] [  226/  468] time: 20.8871, train_loss: 0.22472426, train_accuracy: 0.9375, test_Accuracy: 0.9412
Epoch: [ 0] [  227/  468] time: 20.9786, train_loss: 0.33764786, train_accuracy: 0.9141, test_Accuracy: 0.9426
Epoch: [ 0] [  228/  468] time: 21.0626, train_loss: 0.17863293, train_accuracy: 0.9688, test_Accuracy: 0.9445
Epoch: [ 0] [  229/  468] time: 21.1489, train_loss: 0.23461325, train_accuracy: 0.9453, test_Accuracy: 0.9448
Epoch: [ 0] [  230/  468] time: 21.2325, train_loss: 0.24174951, train_accuracy: 0.9297, test_Accuracy: 0.9430
Epoch: [ 0] [  231/  468] time: 21.3200, train_loss: 0.23720506, train_accuracy: 0.9609, test_Accuracy: 0.9438
Epoch: [ 0] [  232/  468] time: 21.4056, train_loss: 0.22929281, train_accuracy: 0.9531, test_Accuracy: 0.9429
Epoch: [ 0] [  233/  468] time: 21.4944, train_loss: 0.19252551, train_accuracy: 0.9375, test_Accuracy: 0.9380
E

Epoch: [ 0] [  300/  468] time: 27.3927, train_loss: 0.13398191, train_accuracy: 0.9609, test_Accuracy: 0.9467
Epoch: [ 0] [  301/  468] time: 27.4805, train_loss: 0.20146182, train_accuracy: 0.9375, test_Accuracy: 0.9460
Epoch: [ 0] [  302/  468] time: 27.5662, train_loss: 0.15448633, train_accuracy: 0.9609, test_Accuracy: 0.9479
Epoch: [ 0] [  303/  468] time: 27.6577, train_loss: 0.25520337, train_accuracy: 0.9453, test_Accuracy: 0.9503
Epoch: [ 0] [  304/  468] time: 27.7433, train_loss: 0.07880191, train_accuracy: 0.9766, test_Accuracy: 0.9492
Epoch: [ 0] [  305/  468] time: 27.8278, train_loss: 0.08835080, train_accuracy: 0.9766, test_Accuracy: 0.9478
Epoch: [ 0] [  306/  468] time: 27.9211, train_loss: 0.15020545, train_accuracy: 0.9531, test_Accuracy: 0.9485
Epoch: [ 0] [  307/  468] time: 28.0069, train_loss: 0.26754624, train_accuracy: 0.9375, test_Accuracy: 0.9509
Epoch: [ 0] [  308/  468] time: 28.0948, train_loss: 0.22444752, train_accuracy: 0.9375, test_Accuracy: 0.9525
E

Epoch: [ 0] [  375/  468] time: 33.9995, train_loss: 0.09185401, train_accuracy: 0.9844, test_Accuracy: 0.9566
Epoch: [ 0] [  376/  468] time: 34.1128, train_loss: 0.12932323, train_accuracy: 0.9609, test_Accuracy: 0.9546
Epoch: [ 0] [  377/  468] time: 34.2246, train_loss: 0.15024242, train_accuracy: 0.9688, test_Accuracy: 0.9538
Epoch: [ 0] [  378/  468] time: 34.3113, train_loss: 0.08063377, train_accuracy: 0.9844, test_Accuracy: 0.9535
Epoch: [ 0] [  379/  468] time: 34.3992, train_loss: 0.13662067, train_accuracy: 0.9688, test_Accuracy: 0.9538
Epoch: [ 0] [  380/  468] time: 34.4869, train_loss: 0.11290300, train_accuracy: 0.9688, test_Accuracy: 0.9530
Epoch: [ 0] [  381/  468] time: 34.5802, train_loss: 0.14997181, train_accuracy: 0.9609, test_Accuracy: 0.9529
Epoch: [ 0] [  382/  468] time: 34.6697, train_loss: 0.17041759, train_accuracy: 0.9531, test_Accuracy: 0.9523
Epoch: [ 0] [  383/  468] time: 34.7545, train_loss: 0.18390323, train_accuracy: 0.9531, test_Accuracy: 0.9496
E

Epoch: [ 0] [  451/  468] time: 40.7736, train_loss: 0.08190832, train_accuracy: 0.9844, test_Accuracy: 0.9615
Epoch: [ 0] [  452/  468] time: 40.8602, train_loss: 0.07666154, train_accuracy: 0.9688, test_Accuracy: 0.9621
Epoch: [ 0] [  453/  468] time: 40.9497, train_loss: 0.05275073, train_accuracy: 0.9844, test_Accuracy: 0.9622
Epoch: [ 0] [  454/  468] time: 41.0362, train_loss: 0.15831397, train_accuracy: 0.9766, test_Accuracy: 0.9626
Epoch: [ 0] [  455/  468] time: 41.1188, train_loss: 0.15586478, train_accuracy: 0.9531, test_Accuracy: 0.9617
Epoch: [ 0] [  456/  468] time: 41.2008, train_loss: 0.06750786, train_accuracy: 0.9922, test_Accuracy: 0.9613
Epoch: [ 0] [  457/  468] time: 41.2892, train_loss: 0.10472396, train_accuracy: 0.9844, test_Accuracy: 0.9608
Epoch: [ 0] [  458/  468] time: 41.3859, train_loss: 0.14222562, train_accuracy: 0.9531, test_Accuracy: 0.9598
Epoch: [ 0] [  459/  468] time: 41.4767, train_loss: 0.16641076, train_accuracy: 0.9375, test_Accuracy: 0.9592
E

###weight Initialization

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [12]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [13]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### 모델 함수

In [15]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### class.V

In [16]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #he_uniform

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

### func.V

In [20]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [21]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [22]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [23]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Success to read nn_xavier-468-1
 [*] Load SUCCESS


###Dropout

In [26]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) #true : dropout 사용하겟다는뜻
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [24]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

In [25]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [37]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    #global_step = tf.compat.v1.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Success to read nn_xavier-468-1
 [*] Load SUCCESS


AttributeError: module 'tensorboard.summary._tf.summary' has no attribute 'always_record_summaries'

###Batch Normalization

In [27]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [28]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x